###TESTING OUT THE MNIST DATASET (KAGGLE)

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow import keras
import pandas as pd

In [ ]:
(X_train, Y_train), (_, _) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
test_data = pd.read_csv("/content/sample_data/test.csv")

In [ ]:
X_test = test_data.values

print("X_train shape: {0}".format(X_train.shape))
print("Y_train shape: {0} ".format(Y_train.shape))
print("X_test shape: {0}".format(X_test.shape))

X_train shape: (60000, 28, 28)
Y_train shape: (60000,) 
X_test shape: (28000, 784)


In [ ]:
def normalize_data(data):
  return data / 255.0

In [ ]:
X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

print("Maximum values in training set: {0}".format(np.max(X_train)))
print("Maximum values in test set: {0}".format(np.max(X_test)))

Maximum values in training set: 1.0
Maximum values in test set: 1.0


In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))

Do not normalize data anymore!!!

###Neural Network

In [ ]:
class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()
    self.patience = patience
    self.best_weights = None

  def on_train_begin(self, logs=None):
    self.wait = 0
    self.stopped_epoch = 0
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get("loss")
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print("Restoring model weights from the end of the best epoch.")
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

In [ ]:
num_labels = int(np.max(Y_train + 1))

nn_model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (5,5), strides=2, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),

    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (5,5), strides=2, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),

    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(num_labels, activation='softmax')
])

nn_model.summary()

nn_model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=lr_schedule),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_48 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 batch_normalization_66 (Bat  (None, 26, 26, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_49 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 batch_normalization_67 (Bat  (None, 24, 24, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_50 (Conv2D)          (None, 12, 12, 32)        25632     
                                                                 
 batch_normalization_68 (Bat  (None, 12, 12, 32)      

In [ ]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

In [ ]:
hist = nn_model.fit(x=X_train, y=Y_train, epochs=30, batch_size=64)

Epoch 1/30
938/938 [==============================] - 48s 8ms/step - loss: 0.2813 - accuracy: 0.9233
Epoch 2/30
938/938 [==============================] - 7s 8ms/step - loss: 0.1367 - accuracy: 0.9618
Epoch 3/30
938/938 [==============================] - 7s 8ms/step - loss: 0.1176 - accuracy: 0.9678
Epoch 4/30
938/938 [==============================] - 7s 8ms/step - loss: 0.1046 - accuracy: 0.9718
Epoch 5/30
938/938 [==============================] - 8s 8ms/step - loss: 0.0906 - accuracy: 0.9762
Epoch 6/30
938/938 [==============================] - 8s 8ms/step - loss: 0.0829 - accuracy: 0.9786
Epoch 7/30
938/938 [==============================] - 8s 8ms/step - loss: 0.0769 - accuracy: 0.9805
Epoch 8/30
938/938 [==============================] - 7s 8ms/step - loss: 0.0653 - accuracy: 0.9830
Epoch 9/30
938/938 [==============================] - 7s 8ms/step - loss: 0.0653 - accuracy: 0.9832
Epoch 10/30
938/938 [==============================] - 7s 8ms/step - loss: 0.0597 - accuracy: 0.985

In [ ]:
Y_predict = nn_model.predict(X_test)
Y_classes = np.argmax(Y_predict, axis=1)

In [ ]:
Y_classes

array([2, 0, 9, ..., 3, 9, 2])

In [ ]:
submission = pd.DataFrame({"ImageId": list(range(1, len(Y_classes) + 1)),
                           "Label": Y_classes})

In [ ]:
submission.to_csv("/content/sample_data/cnn_submission.csv", index=False, header=True)